In [2]:
import cv2
import numpy as np
import glob

from matplotlib import pyplot as plt

images = glob.glob("./images/checkerboard_*.jpeg")

In [19]:
CHECKERBOARD = (7, 9)
SQUARE_SIZE = 20.0

objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[1], 0:CHECKERBOARD[0]].T.reshape(-1, 2)
objp *= SQUARE_SIZE  # Scale by square size

objpoints = []
imgpoints = []

focal_length = 2.1
pixel_size = 0.003

camera_matrix_initial = np.array(
  [
    [focal_length, 0, 640],
    [0, focal_length, 360],
    [0, 0, 1]
  ], dtype=np.float32
)

In [20]:
for fname in images:
  img = cv2.imread(fname)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

  if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    objpoints.append(objp)
    imgpoints.append(corners2)

    cv2.drawChessboardCorners(img, (11, 11), corners2, ret)
    # plt.imshow(img)
    # plt.show()
  else:
    print(fname)


In [21]:
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], camera_matrix_initial, None, flags=cv2.CALIB_USE_INTRINSIC_GUESS)
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs.ravel())

Camera Matrix:
 [[  2.0999999   0.        640.       ]
 [  0.          2.0999999 360.       ]
 [  0.          0.          1.       ]]
Distortion Coefficients:
 [ 1.40017137e-14  2.14617695e-10 -1.38004061e-16  2.35596742e-16
 -6.64757209e-15]
